In [12]:
# references:
# https://github.com/Cledge-org/cledge/blob/dev/features/college_search_tool/college_picture_scraper/src/college_picture_url_scraper.ipynb

# set up azure virtual machine Linux system desktop:
# https://docs.microsoft.com/en-us/azure/virtual-machines/linux/use-remote-desktop?tabs=azure-cli

import time
import json

from pyspark.sql import *
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# Variables
DRIVER_PATH = './driver/chromedriver'
DATA_PATH = '../data/college-search-data-v2.parquet'
TARGET_FILE = './college_picture_urls.json'

In [3]:
# Initialize SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
# Load data
df = spark.read.load(DATA_PATH)

In [5]:
# Collect list of college names
colleges = {}
for row in df.select(["UNITID", "INSTNM"]).collect():
    colleges[row.UNITID] = ' '.join(c for c in row.INSTNM.replace('-', ' ').split() if c.isalnum())

In [110]:
def fetch_image_data(college_name_str, wd, sleep=0.5):
    # search limit to wikipedia and wikimedia along with creative commons restriction
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&tbs=il:cl&source=hp&q=site:wikipedia.org, commons.wikimedia.org {q}&oq=site:wikipedia.org, commons.wikimedia.org {q}&gs_l=img"
    college_name_url_str = college_name_str.lower()
    if 'campus' in college_name_url_str:
        query_str = college_name_url_str
    else:
        query_str = '{} campus'.format(college_name_url_str)
    wd.get(search_url.format(q=query_str))
    thumbnail_results = wd.find_elements(by=By.CSS_SELECTOR, value="img.Q4LuWd")

    result_obj = {
        'college_name': college_name_str,
        'img_title': None,
        'img_wiki_link': None,
        'img_link': None,
        'img_description': None
    }

    for res in thumbnail_results:
        try:
            res.click()
        except Exception:
            continue

        time.sleep(sleep)

        try:
            imgs = wd.find_elements(by=By.CSS_SELECTOR, value="img.n3VNCb") # finds actual image link
            for img_element in imgs:
                src = img_element.get_attribute('src')
                if src and src.startswith('http'):
                    result_obj['img_link'] = src
                    break
            if not result_obj['img_link']:
                continue
        except Exception:
            result_obj['img_link'] = None
            continue
        try:
            img_title_element = wd.find_elements(by=By.CSS_SELECTOR, value="h1.eYbsle.mKq8g.Da4KQe") # get image title
            result_obj['img_title'] = img_title_element[0].text
        except Exception:
            result_obj['img_title'] = None
            continue
        try:
            links = wd.find_elements(by=By.CSS_SELECTOR, value="a.zSA7pe.goedYd")
            for link_element in links:
                if 'wiki' in link_element.text.lower():
                    result_obj['img_wiki_link'] = link_element.get_attribute('href')
                    print(result_obj['img_wiki_link'])
                    # wd.execute_script('''window.open("{}","_blank");'''.format(result_obj['img_wiki_link']))
                    # wd.switch_to_window(wd.window_handles[1])
                    wd.switch_to.new_window("tab")
                    wd.get(result_obj['img_wiki_link'])
                    try:
                        desc_element = wd.find_elements(by=By.CSS_SELECTOR, value="div.description.mw-content-ltr.en")
                    except Exception:
                        try:
                            desc_element = wd.find_elements(by=By.CSS_SELECTOR, value="td.description")
                        except Exception:
                            try:
                                desc_element = wd.find_elements(by=By.CSS_SELECTOR, value="div.description.mw-content-ltr.en a.extiw")
                            except Exception:
                                desc_element = None
                    if desc_element:
                        result_obj['img_description'] = desc_element[0].text
                    else:
                        result_obj['img_description'] = None
                    break
        except Exception:
            result_obj['img_wiki_link'] = None
            result_obj['img_description'] = None
        break

    return result_obj

In [111]:
# Testing code block
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
print(fetch_image_data('University of Washington Seattle Campus', wd))
wd.quit()

/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


https://commons.wikimedia.org/wiki/File:University_of_Washington_.jpg
{'college_name': 'University of Washington Seattle Campus', 'img_title': 'File:University of Washington .jpg - Wikimedia Commons', 'img_wiki_link': 'https://commons.wikimedia.org/wiki/File:University_of_Washington_.jpg', 'img_link': 'https://upload.wikimedia.org/wikipedia/commons/b/b0/University_of_Washington_.jpg', 'img_description': 'English: George Washington with view of Red Square in the background'}


In [12]:
with open(TARGET_FILE) as f:
    pic_urls = json.load(f)

In [13]:
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
for UNITID, INSTNM in colleges.items():
    if UNITID in pic_urls:
        continue
    pic_urls[UNITID] = fetch_image_data(INSTNM, wd)
    with open(TARGET_FILE, 'w') as f:
        f.write(json.dumps(pic_urls, indent=4))
wd.quit()

/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  # Remove the CWD from sys.path while we load stuff.
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykern

KeyboardInterrupt: 